In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.use('agg')
from matplotlib.figure import Figure
import panel as pn
import statistics
import math
import param
import io
import os
from panel.viewable import Viewer
from scipy import stats
import base64

In [22]:
data = pd.read_excel('/Users/ctlewis/U-Pb-LA-ICP-MS-Reduction/output_lasertramZ_one_010523_noc.xlsx',
                         sheet_name='Sheet1')
data['U238_Pb206'] = 1/data['206/238 2nd Order']
data['Pb207_Pb206'] = data['207Pb/206Pb']

In [23]:
# define constants for reducing U-Pb data
lambda_238 = 1.55125e-10 #
lambda_238err = 1.55125e-10*0.16 #
lambda_235 = 9.8485e-10 #
lambda_232 = 4.9475e-11 # 

In [24]:
# calculate initial ages and Stacey-Kramers Common Pb for that age
data['206Pb/238U_age_init'] = np.log(data['206/238 2nd Order'] + 1) / lambda_238

data['SK207_204'] = 12.998 + 9.74/137.82*(np.exp(lambda_235*3.7e9)-np.exp(lambda_235*data['206Pb/238U_age_init']))
data['SK206_204'] = 11.152 + 9.74*(np.exp(lambda_238*3.7e9)-np.exp(lambda_238*data['206Pb/238U_age_init']))
data['SK207_206'] = data['SK207_204'] / data['SK206_204']


In [25]:
# # calculate initial ages and Stacey-Kramers Common Pb for that age
# data['206Pb/238Uc_age_init'] = np.log(data['206/238 2nd Order'] + 1) / lambda_238

# data['SK207_204'] = 12.998 + 9.74/137.82*(np.exp(lambda_235*3.7e9)-np.exp(lambda_235*data['206Pb/238Uc_age_init']))
# data['SK206_204'] = 11.152 + 9.74*(np.exp(lambda_238*3.7e9)-np.exp(lambda_238*data['206Pb/238Uc_age_init']))
# # data['SK208_204'] = 31.230 + 37.4*(np.exp(lambda_232*3.7e9)-np.exp(lambda_232*data['206Pb/238Uc_age_init']))

# # data['SK_slope_calc'] = (data['SK207_204'] - 12.998) / (data['SK206_204'] - 11.152)

# # data['counts_pb206'] = data['238U'] * data['206/238 2nd Order']
# data['SK207_206'] = data['SK207_204'] / data['SK206_204']
# # data['U238_Pb206c'] = 1 / data['206Pb/238Uc']

In [26]:
# samples = data.groupby(['SampleLabel'])
# samples.get_group('MADER') # put this in selector to go sample by sample

In [27]:
# # function block

# # this needs to be a function that receives the selected sample
# def get_data_TW_regressions(df,select_regression):
#     """
#     """
#     if select_regression == '1st Order':
#         data['U238_Pb206'] = 1/data['206/238 1st Order']
#         data['206Pb/238U_age_init'] = np.log(data['206/238 1st Order'] + 1) / lambda_238
#         data['SK207_204'] = 12.998 + 9.74/137.82*(np.exp(lambda_235*3.7e9)-np.exp(lambda_235*data['206Pb/238U_age_init']))
#         data['SK206_204'] = 11.152 + 9.74*(np.exp(lambda_238*3.7e9)-np.exp(lambda_238*data['206Pb/238U_age_init']))
#         data['SK207_206'] = data['SK207_204'] / data['SK206_204']
#         pts_x = np.array([df.U238_Pb206,np.zeros_like(df['SK207_206'])]).T
#         pts_y = np.array([df['207Pb/206Pb'],df['SK207_206']]).T
#         discordia_t = np.zeros((len(df),2))

#         for i in range(0,len(df)):
#             discordia_t[i] = np.poly1d(np.polyfit(pts_x[i],pts_y[i],1))
#     elif select_regression == '2nd Order':
#         data['U238_Pb206'] = 1/data['206/238 2nd Order']
#         data['206Pb/238U_age_init'] = np.log(data['206/238 2nd Order'] + 1) / lambda_238
#         data['SK207_204'] = 12.998 + 9.74/137.82*(np.exp(lambda_235*3.7e9)-np.exp(lambda_235*data['206Pb/238U_age_init']))
#         data['SK206_204'] = 11.152 + 9.74*(np.exp(lambda_238*3.7e9)-np.exp(lambda_238*data['206Pb/238U_age_init']))
#         data['SK207_206'] = data['SK207_204'] / data['SK206_204']
#         pts_x = np.array([df.U238_Pb206,np.zeros_like(df['SK207_206'])]).T
#         pts_y = np.array([df['207Pb/206Pb'],df['SK207_206']]).T
#         discordia_t = np.zeros((len(df),2))

#         for i in range(0,len(df)):
#             discordia_t[i] = np.poly1d(np.polyfit(pts_x[i],pts_y[i],1))

#     return discordia_t



# def get_TW_concordia():
#     """
#     """
#     # calculate T-W discordia - put in plot function
#     t = np.linspace(1,4.6e9,100000)
#     pb207_206 = np.zeros(len(t))
#     pb206 = np.zeros(len(t))
#     u238_pb206 = np.zeros(len(t))
#     pb207_206r = np.zeros(len(t))
#     for i in range(0,len(t)):
#         u238_pb206[i] = 1/(np.exp(lambda_238*t[i])-1)
#         pb207_206r[i] = (1/137.88) * ((np.exp(lambda_235*t[i])-1) / (np.exp(lambda_238*t[i])-1))
        
#     return u238_pb206,pb207_206r




# def get_projections(df,select_regression):
#     """
#     """
#     # get the TW concordia values
#     x_TW,y_TW = get_TW_concordia()
#     discorida_regressions = get_data_TW_regressions(df,select_regression) # get regressions
#     # array of xvalues to project over
#     x_vals = np.linspace(min(x_TW),max(x_TW),100000)
#     # pts_pb_r = [] # set up array to be filled with calculated radiogenic lead component
#     pts_pb_r = np.zeros(len(discorida_regressions))
#     concordia_238_206 = np.zeros(len(discorida_regressions))

#     for i in range(0,len(discorida_regressions)):

#         discordia_207206 = discorida_regressions[i][0]*x_vals+discorida_regressions[i][1]
#         discordia_238206 = (discordia_207206-discorida_regressions[i][1])/discorida_regressions[i][0]

#         delta_y = (discorida_regressions[i][1] + x_TW * discorida_regressions[i][0]) - y_TW # distance of y value from line
#         indx = np.where(delta_y[1:]*delta_y[:-1]<0)[0] # index the regression for where the curve cross the regression
#         d_ratio = delta_y[indx] / (delta_y[indx] - delta_y[indx + 1]) # similar triangles geometry gets points
#         points = np.zeros((len(indx), 2)) # empty array for crossing points
#         points[:,0] = x_TW[indx] + d_ratio * (x_TW[indx+1] - x_TW[indx]) # x crossings
#         points[:,1] = y_TW[indx] + d_ratio * (y_TW[indx+1] - y_TW[indx]) # y crossings
#         y_point = y_TW[indx] + d_ratio * (y_TW[indx+1] - y_TW[indx]) # y crossings
#         x_point = x_TW[indx] + d_ratio * (x_TW[indx+1] - x_TW[indx]) # x crossings
        
#         if len(y_point) >= 1:
#             pts_pb_r[i] = min(y_point)
#         elif len(y_point) < 0:
#             pts_pb_r[i] = 0
            
#         # need to work this out to get concordia points, then use these to calculate ages, store other ages as well.
#         # may need to fuck with delta_y and make a delta_x, etc
#         if len(x_point) > 1:
#             concordia_238_206[i] = min(x_point)
#         elif len(x_point) == 1:
#             concordia_238_206[i] = x_point
    
#     return points,concordia_238_206,pts_pb_r,discordia_207206,discordia_238206

    

# def plot_TW(df,xlim_start,xlim_stop,ylim_start,ylim_stop,select_regression):
#     """
#     """
#     plt.style.use('seaborn-colorblind')
#     # fig = Figure(figsize=(4,7))
#     fig = Figure()
#     ax = fig.add_subplot()
    
#     x_TW,y_TW = get_TW_concordia()
    
#     disc_regressions = get_data_TW_regressions(df,select_regression)
    
#     x_vals = np.linspace(min(x_TW),max(x_TW),100000)
#     pts_pb_r = np.zeros(len(disc_regressions))
    
#     for i in range(0,len(disc_regressions)):
#         # points,pts_pb_r,discordia_207206,discordia_238206 = get_projections(df)
#         discordia_207206 = disc_regressions[i][0]*x_vals+disc_regressions[i][1]
#         discordia_238206 = (discordia_207206-disc_regressions[i][1])/disc_regressions[i][0]

#         delta_y = (disc_regressions[i][1] + x_TW * disc_regressions[i][0]) - y_TW # distance of y value from line
#         indx = np.where(delta_y[1:]*delta_y[:-1]<0)[0] # index the regression for where the curve cross the regression
#         d_ratio = delta_y[indx] / (delta_y[indx] - delta_y[indx + 1]) # similar triangles geometry gets points
#         points = np.zeros((len(indx), 2)) # empty array for crossing points
#         points[:,0] = x_TW[indx] + d_ratio * (x_TW[indx+1] - x_TW[indx]) # x crossings
#         points[:,1] = y_TW[indx] + d_ratio * (y_TW[indx+1] - y_TW[indx]) # y crossings
#         y_point = y_TW[indx] + d_ratio * (y_TW[indx+1] - y_TW[indx]) # y crossings
#         if len(y_point) >= 1:
#             pts_pb_r[i] = min(y_point)
#         elif len(y_point) < 0:
#             pts_pb_r[i] = 0
            
#         ax.plot(discordia_238206,discordia_207206,'-k',lw=0.5)
#         ax.plot(points[:,0],points[:,1],'og',lw=1,mec='k')
        
#     ax.plot(df.U238_Pb206,df.Pb207_Pb206,'kd')
#     if select_regression == '1st Order':
#         ax.errorbar(df.U238_Pb206,df.Pb207_Pb206,df['206/238 1st Order'],df['207Pb/206Pb'],fmt='none',ecolor='k',elinewidth=0.5)
#     elif select_regression == '2nd Order':
#         ax.errorbar(df.U238_Pb206,df.Pb207_Pb206,df['206/238 2nd Order'],df['207Pb/206Pb'],fmt='none',ecolor='k',elinewidth=0.5)
#     else:
#         pass
#     ax.plot(x_TW,y_TW,'k',lw=1)
#     # ax.legend(loc='upper right',fontsize=8)
#     ax.set_xlabel('$^{238}$U / $^{206}$Pb$^{*}$',fontsize=8)
#     ax.set_ylabel('[$^{207}$Pb / $^{206}$Pb]$^{*}$',fontsize=8)
#     ax.set_ylim(ylim_start,ylim_stop)
#     ax.set_xlim(x_vals[0],xlim_stop)
#     return fig



# def correct_standard_ages(df,select_regression):
#     """
#     function to calculate Pb and fractionation factor corrected ages
#     """
#     # get points needed for correction
#     points,concordia_238_206,pts_pb_r,na,naII = get_projections(df,select_regression)
    
#     # set up empty arrays to be filled for common lead corrections
#     common_filter = []
#     f_filter = []
    
#     pb_m = df['207Pb/206Pb'] # measured 207/206
#     common = df['SK207_206']
    
#     for i in common:
#         if i <= 0:
#             common_filter.append(0)
#         else:
#             common_filter.append(i)
    
#     # calculate fraction of common Pb
#     f_ = (pb_m - pts_pb_r) / (common - pts_pb_r)
#     # set up array to set f = 0 if point lies on or below Concordia (i.e., no common Pb present)
#     f = []
    
#     for k,j in zip(common_filter,f_):
#         if k <= 0:
#             f.append(0)
#         elif j < 0:
#             f.append(0)
#         else:
#             f.append(j)
            
#     # append the calculations to the sample dataframe
#     df['207Pb/206Pbr'] = pts_pb_r
#     # df['207Pb/206Pbcommon'] = common_filter
#     df['f'] = f
    
#     df['counts_pb206r'] = df['206Pb'] * (1-df['f'])
#     df['206Pb/238Upb'] = 1/concordia_238_206
#     df['206Pb/238U_age'] = np.log(df['206Pb/238Upb'] + 1) / lambda_238
    
#     avg_std_age = df['206Pb/238U_age'].mean()
#     if select_regression == '1st Order':
#         avg_reg_err = df['SE 206/238 1st Order'].mean()
#     elif select_regression == '2nd Order':
#         avg_reg_err = df['SE 206/238 2nd Order'].mean()
#     else:
#         pass
    
#     return avg_std_age,avg_reg_err
    
    

# def correct_sample_ages(df,std,std_txt,select_regression):
#     """
#     function to calculate Pb and fractionation factor corrected ages
#     """
#     # df = df.reset_index()
#     # get points needed for correction
#     points,concordia_238_206,pts_pb_r,n,a = get_projections(df,select_regression)
    
#     # set up empty arrays to be filled for common lead corrections
#     common_filter = []
#     f_filter = []
    
#     pb_m = df['207Pb/206Pb'] # measured 207/206
#     common = df['SK207_206']
    
#     for i in common:
#         if i <= 0:
#             common_filter.append(0)
#         else:
#             common_filter.append(i)
    
#     # calculate fraction of common Pb
#     f_ = (pb_m - pts_pb_r) / (common - pts_pb_r)
#     # set up array to set f = 0 if point lies on or below Concordia (i.e., no common Pb present)
#     f = []
    
#     for k,j in zip(common_filter,f_):
#         if k <= 0:
#             f.append(0)
#         elif j < 0:
#             f.append(0)
#         else:
#             f.append(j)
            
#     # append the calculations to the sample dataframe
#     df['207Pb/206Pbr'] = pts_pb_r
#     # df['207Pb/206Pbcommon'] = common_filter
#     df['f'] = f
    
#     df['counts_pb206r'] = df['206Pb'] * (1-df['f'])
#     # df['206Pb/238Upbc'] = df.counts_pb206r/df['238U']
#     df['206Pb/238Upb'] = 1/concordia_238_206
#     df['206Pb/238U_age'] = np.log(df['206Pb/238Upb'] + 1) / lambda_238

    
#     # will need input for which standard to use now! standard will need to be calculated by a separate function and called/input here.
#     frac_factor,tims_age,tims_error,std_avg,std_avgerr = get_standard_fracfctr(std,std_txt)
#     df['206Pb/238U_correctedage'] = df['206Pb/238Uc_age']*frac_factor
    
#     #propagate errors
#     #error on fractionation factor. includes error from ID-TIMS and ICPMS
#     dfrac = np.abs(frac_factor)*((1/tims_age)**2*tims_error**2 + (1/std_avg)**2*std_avgerr**2)**(1/2)
#     #error on age equation. error on decay constant includes 1.5* counting stats (Mattinson 1987)
#     if select_regression == '1st Order':
#         dage = np.abs(df['206Pb/238U_age'])*((1/df['206/238 1st Order'])**2*df['SE 206/238 1st Order']**2 + (0.16/100)**2 + (1/frac_factor)**2*dfrac**2)**(1/2)
#     elif select_regression == '2nd Order':
#         dage = np.abs(df['206Pb/238U_age'])*((1/df['206/238 2nd Order'])**2*df['SE 206/238 2nd Order']**2 + (0.16/100)**2 + (1/frac_factor)**2*dfrac**2)**(1/2)
#     else:
#         pass
#     #error on estimation for common lead using 207 method. Uses conservaitve estimates of 1.0 for 206/204 and 0.3 for 207/204 (Mattionson, 1987)
#     dinit = np.abs(common)*((1/df['SK207_204'])**2*0.3**2 + (1/df['SK206_204'])**2*1**2)**(1/2)
#     # total propagated error
#     dagetot = (dage**2 +  df['SE 206/204']**2 + dinit**2)**(1/2)
    
#     df['∆206/238 age (meas.)'] = dage
#     df['∆206/238 age (tot.)'] = dagetot
    
#     return df




# def get_standard_fracfctr(std,std_txt,select_regression):
#     """
#     """
#     # selected_std = samples.get_group(std)
    
#     accepted_ages = {
#         'Temora':416780000,
#         'FishCanyon': 28478000,
#         '94_35': 55500000,
#         'Plesovice': 337100000,
#         'R33': 419300000,
#         '91500': 1062400000,
#         'FC1': 1099500000,
#         'Oracle': 1436200000,
#         'Tan-Bra': 2507800000,
#         'OG1': 3440700000
#     }
    
#     TIMS_errors = {
#         'Temora':330000,
#         'FishCanyon': 24000,
#         '94_35': 80000,
#         'Plesovice': 200000,
#         'R33': 400000,
#         '91500': 1900000,
#         'FC1': 330000,
#         'Oracle': 1300000,
#         'Tan-Bra': 1500000,
#         'OG1': 3200000
#     }
    
#     std_avg,std_avgerr = correct_standard_ages(std,select_regression)
#     frac_factor = accepted_ages.get(std_txt)/std_avg
#     tims_age = accepted_ages.get(std_txt)
#     tims_error = TIMS_errors.get(std_txt)
    
#     return frac_factor,tims_age,tims_error,std_avg,std_avgerr

In [28]:
# function block

# this needs to be a function that receives the selected sample
def get_data_TW_regressions(df):
    """
    """
    pts_x = np.array([df.U238_Pb206,np.zeros_like(df['SK207_206'])]).T
    pts_y = np.array([df['207Pb/206Pb'],df['SK207_206']]).T
    discordia_t = np.zeros((len(df),2))

    for i in range(0,len(df)):
        discordia_t[i] = np.poly1d(np.polyfit(pts_x[i],pts_y[i],1))

    return discordia_t



def get_TW_concordia():
    """
    """
    # calculate T-W discordia - put in plot function
    t = np.linspace(1,4.6e9,100000)
    pb207_206 = np.zeros(len(t))
    pb206 = np.zeros(len(t))
    u238_pb206 = np.zeros(len(t))
    pb207_206r = np.zeros(len(t))
    for i in range(0,len(t)):
        u238_pb206[i] = 1/(np.exp(lambda_238*t[i])-1)
        pb207_206r[i] = (1/137.88) * ((np.exp(lambda_235*t[i])-1) / (np.exp(lambda_238*t[i])-1))
        
    return u238_pb206,pb207_206r




def get_projections(df):
    """
    """
    # get the TW concordia values
    x_TW,y_TW = get_TW_concordia()
    discorida_regressions = get_data_TW_regressions(df) # get regressions
    # array of xvalues to project over
    x_vals = np.linspace(min(x_TW),max(x_TW),100000)
    # pts_pb_r = [] # set up array to be filled with calculated radiogenic lead component
    pts_pb_r = np.zeros(len(discorida_regressions))
    concordia_238_206 = np.zeros(len(discorida_regressions))

    for i in range(0,len(discorida_regressions)):

        discordia_207206 = discorida_regressions[i][0]*x_vals+discorida_regressions[i][1]
        discordia_238206 = (discordia_207206-discorida_regressions[i][1])/discorida_regressions[i][0]

        delta_y = (discorida_regressions[i][1] + x_TW * discorida_regressions[i][0]) - y_TW # distance of y value from line
        indx = np.where(delta_y[1:]*delta_y[:-1]<0)[0] # index the regression for where the curve cross the regression
        d_ratio = delta_y[indx] / (delta_y[indx] - delta_y[indx + 1]) # similar triangles geometry gets points
        points = np.zeros((len(indx), 2)) # empty array for crossing points
        points[:,0] = x_TW[indx] + d_ratio * (x_TW[indx+1] - x_TW[indx]) # x crossings
        points[:,1] = y_TW[indx] + d_ratio * (y_TW[indx+1] - y_TW[indx]) # y crossings
        y_point = y_TW[indx] + d_ratio * (y_TW[indx+1] - y_TW[indx]) # y crossings
        x_point = x_TW[indx] + d_ratio * (x_TW[indx+1] - x_TW[indx]) # x crossings
        
        if len(y_point) >= 1:
            pts_pb_r[i] = min(y_point)
        elif len(y_point) < 0:
            pts_pb_r[i] = 0
            
        # need to work this out to get concordia points, then use these to calculate ages, store other ages as well.
        # may need to fuck with delta_y and make a delta_x, etc
        if len(x_point) > 1:
            concordia_238_206[i] = min(x_point)
        elif len(x_point) == 1:
            concordia_238_206[i] = x_point
    
    return points,concordia_238_206,pts_pb_r,discordia_207206,discordia_238206

    

def plot_TW(df,xlim_start,xlim_stop,ylim_start,ylim_stop):
    """
    """
    plt.style.use('seaborn-colorblind')
    # fig = Figure(figsize=(4,7))
    fig = Figure()
    ax = fig.add_subplot()
    
    x_TW,y_TW = get_TW_concordia()
    
    disc_regressions = get_data_TW_regressions(df)
    
    x_vals = np.linspace(min(x_TW),max(x_TW),100000)
    pts_pb_r = np.zeros(len(disc_regressions))
    
    for i in range(0,len(disc_regressions)):
        # points,pts_pb_r,discordia_207206,discordia_238206 = get_projections(df)
        discordia_207206 = disc_regressions[i][0]*x_vals+disc_regressions[i][1]
        discordia_238206 = (discordia_207206-disc_regressions[i][1])/disc_regressions[i][0]

        delta_y = (disc_regressions[i][1] + x_TW * disc_regressions[i][0]) - y_TW # distance of y value from line
        indx = np.where(delta_y[1:]*delta_y[:-1]<0)[0] # index the regression for where the curve cross the regression
        d_ratio = delta_y[indx] / (delta_y[indx] - delta_y[indx + 1]) # similar triangles geometry gets points
        points = np.zeros((len(indx), 2)) # empty array for crossing points
        points[:,0] = x_TW[indx] + d_ratio * (x_TW[indx+1] - x_TW[indx]) # x crossings
        points[:,1] = y_TW[indx] + d_ratio * (y_TW[indx+1] - y_TW[indx]) # y crossings
        y_point = y_TW[indx] + d_ratio * (y_TW[indx+1] - y_TW[indx]) # y crossings
        if len(y_point) >= 1:
            pts_pb_r[i] = min(y_point)
        elif len(y_point) < 0:
            pts_pb_r[i] = 0
            
        ax.plot(discordia_238206,discordia_207206,'-k',lw=0.5)
        ax.plot(points[:,0],points[:,1],'og',lw=1,mec='k')
        
    ax.plot(df.U238_Pb206,df.Pb207_Pb206,'kd')
    ax.errorbar(df.U238_Pb206,df.Pb207_Pb206,df['206/238 2nd Order'],df['207Pb/206Pb'],fmt='none',ecolor='k',elinewidth=0.5)
    ax.plot(x_TW,y_TW,'k',lw=1)
    # ax.legend(loc='upper right',fontsize=8)
    ax.set_xlabel('$^{238}$U / $^{206}$Pb$^{*}$',fontsize=8)
    ax.set_ylabel('[$^{207}$Pb / $^{206}$Pb]$^{*}$',fontsize=8)
    ax.set_ylim(ylim_start,ylim_stop)
    ax.set_xlim(x_vals[0],xlim_stop)
    return fig



def correct_standard_ages(df):
    """
    function to calculate Pb and fractionation factor corrected ages
    """
    # get points needed for correction
    points,concordia_238_206,pts_pb_r,na,naII = get_projections(df)
    
    # set up empty arrays to be filled for common lead corrections
    common_filter = []
    f_filter = []
    
    pb_m = df['207Pb/206Pb'] # measured 207/206
    common = df['SK207_206']
    
    for i in common:
        if i <= 0:
            common_filter.append(0)
        else:
            common_filter.append(i)
    
    # calculate fraction of common Pb
    f_ = (pb_m - pts_pb_r) / (common - pts_pb_r)
    # set up array to set f = 0 if point lies on or below Concordia (i.e., no common Pb present)
    f = []
    
    for k,j in zip(common_filter,f_):
        if k <= 0:
            f.append(0)
        elif j < 0:
            f.append(0)
        else:
            f.append(j)
            
    # append the calculations to the sample dataframe
    df['207Pb/206Pbr'] = pts_pb_r
    # df['207Pb/206Pbcommon'] = common_filter
    df['f'] = f
    
    df['counts_pb206r'] = df['206Pb'] * (1-df['f'])
    df['206Pb/238Upb'] = 1/concordia_238_206
    df['206Pb/238U_age'] = np.log(df['206Pb/238Upb'] + 1) / lambda_238
    
    avg_std_age = df['206Pb/238U_age'].mean()
    avg_reg_err = df['SE 206/238 2nd Order'].mean()
    
    return avg_std_age,avg_reg_err
    
    

def correct_sample_ages(df,std,std_txt):
    """
    function to calculate Pb and fractionation factor corrected ages
    """
    # df = df.reset_index()
    # get points needed for correction
    points,concordia_238_206,pts_pb_r,n,a = get_projections(df)
    
    # set up empty arrays to be filled for common lead corrections
    common_filter = []
    f_filter = []
    
    pb_m = df['207Pb/206Pb'] # measured 207/206
    common = df['SK207_206']
    
    for i in common:
        if i <= 0:
            common_filter.append(0)
        else:
            common_filter.append(i)
    
    # calculate fraction of common Pb
    f_ = (pb_m - pts_pb_r) / (common - pts_pb_r)
    # set up array to set f = 0 if point lies on or below Concordia (i.e., no common Pb present)
    f = []
    
    for k,j in zip(common_filter,f_):
        if k <= 0:
            f.append(0)
        elif j < 0:
            f.append(0)
        else:
            f.append(j)
            
    # append the calculations to the sample dataframe
    df['207Pb/206Pbr'] = pts_pb_r
    # df['207Pb/206Pbcommon'] = common_filter
    df['f'] = f
    
    df['counts_pb206r'] = df['206Pb'] * (1-df['f'])
    # df['206Pb/238Upbc'] = df.counts_pb206r/df['238U']
    df['206Pb/238Upb'] = 1/concordia_238_206
    df['206Pb/238U_age'] = np.log(df['206Pb/238Upb'] + 1) / lambda_238

    
    # will need input for which standard to use now! standard will need to be calculated by a separate function and called/input here.
    frac_factor,tims_age,tims_error,std_avg,std_avgerr = get_standard_fracfctr(std,std_txt)
    df['206Pb/238U_correctedage'] = df['206Pb/238U_age']*frac_factor
    
    #propagate errors
    #error on fractionation factor. includes error from ID-TIMS and ICPMS
    dfrac = np.abs(frac_factor)*((1/tims_age)**2*tims_error**2 + (1/std_avg)**2*std_avgerr**2)**(1/2)
    #error on age equation. error on decay constant includes 1.5* counting stats (Mattinson 1987)
    dage = np.abs(df['206Pb/238U_age'])*((1/df['206/238 2nd Order'])**2*df['SE 206/238 2nd Order']**2 + (0.16/100)**2 + (1/frac_factor)**2*dfrac**2)**(1/2)
    #error on estimation for common lead using 207 method. Uses conservaitve estimates of 1.0 for 206/204 and 0.3 for 207/204 (Mattionson, 1987)
    dinit = np.abs(common)*((1/df['SK207_204'])**2*0.3**2 + (1/df['SK206_204'])**2*1**2)**(1/2)
    # total propagated error
    dagetot = (dage**2 +  df['SE 206/204']**2 + dinit**2)**(1/2)
    
    df['∆206/238 age (meas.)'] = dage
    df['∆206/238 age (tot.)'] = dagetot
    
    return df




def get_standard_fracfctr(std,std_txt):
    """
    """
    # selected_std = samples.get_group(std)
    
    accepted_ages = {
        'Temora':416780000,
        'FishCanyon': 28478000,
        '94_35': 55500000,
        'Plesovice': 337100000,
        'R33': 419300000,
        '91500': 1062400000,
        'FC1': 1099500000,
        'Oracle': 1436200000,
        'Tan-Bra': 2507800000,
        'OG1': 3440700000
    }
    
    TIMS_errors = {
        'Temora':330000,
        'FishCanyon': 24000,
        '94_35': 80000,
        'Plesovice': 200000,
        'R33': 400000,
        '91500': 1900000,
        'FC1': 330000,
        'Oracle': 1300000,
        'Tan-Bra': 1500000,
        'OG1': 3200000
    }
    
    std_avg,std_avgerr = correct_standard_ages(std)
    frac_factor = accepted_ages.get(std_txt)/std_avg
    tims_age = accepted_ages.get(std_txt)
    tims_error = TIMS_errors.get(std_txt)
    
    return frac_factor,tims_age,tims_error,std_avg,std_avgerr

In [29]:
class finalize_ages(param.Parameterized):
    
    # set up parameters. those with the suffix _pass are set up in order to be updated in the first callback below.
    input_data = param.DataFrame(default=data,precedence=-1) # wont need output since writing back to df?
    blank_output = pd.DataFrame([np.zeros(len(data.columns))],columns=list(data.columns))
    output_data = param.DataFrame(default=blank_output,precedence=-0.2)
    x_axis_TW = param.Range(default=(1,100),bounds=(0,4000)) # for x-axis
    y_axis_TW = param.Range(default=(0,1),bounds=(0,3)) # for y-axis
    text_sample_selector = param.String(default='Input Sample ID')# for getting samples. need to split to get all of given name... or have them do text input
    text_standard_selector = param.String(default='Input Standard ID')# for getting samples. need to split to get all of given name... or have them do text input
    update_output_button = param.Action(lambda x: x.add_output_data(),label='Approve Data')
    export_data_button = param.Action(lambda x: x.export_data(),label='DDDT!')
    select_regression = param.ObjectSelector(default='1st Order', objects=['1st Order','2nd Order'])
    
    def __init__(self,**params):
        super().__init__(**params)
        self.input_data_widget = pn.Param(self.param.input_data),
        self.output_data_widget = pn.Param(self.param.output_data),
        self.widgets = pn.Param(self,parameters=['x_axis_TW','y_axis_TW','text_standard_selector','update_output_button','export_data_button'])
        # may need to put in another widgets param
        # self._layout = pn.Row(
        #     pn.Param(self,parameters=['update_output_button','sample_subset','ablation_slider','background_slider','ablation_plot_ylim_slider',
        #                               'ratio_plot_ylim_slider',
        #                               ]
        #             ),
        #     pn.Param(widgets={'ratio_buttons': pn.widgets.CheckBoxGroup, 'regression_buttons': pn.widgets.CheckBoxGroup,
        #                      'export_data_button': pn.widgets.CheckBoxGroup(name='DDDT',button_type='success')}),
        #     self.input_data_widget,
        #     self.output_data_widget,
        #     width=1,
        #     sizing_mode='fixed'
        # )
        
    @pn.depends('output_data',watch=True)
    def _update_data_widget(self):
        self.output_data_widget = self.output_data
        self.output_data_widget.height = 40
        self.output_data_widget.heightpolicy = 'Fixed'
        return pn.widgets.Tabulator(self.output_data_widget,width=200)
        
    @pn.depends('input_data','text_sample_selector','x_axis_TW','y_axis_TW','select_regression')
    def call_TW(self):
        # if self.text_sample_selector is not 'Input Sample ID':
        data_toplot = self.input_data[self.input_data['SampleLabel'].str.contains(self.text_sample_selector)]
        return plot_TW(data_toplot,self.x_axis_TW[0],self.x_axis_TW[1],self.y_axis_TW[0],self.y_axis_TW[1])
    
    # @pn.depends('input_data','text_sample_selector','text_standard_selector')
    def add_output_data(self,event=None):
        #need to add columns to input data somewhere else
        data_to_update = self.input_data[self.input_data['SampleLabel'].str.contains(self.text_sample_selector)]
        chosen_std = self.input_data[self.input_data['SampleLabel'].str.contains(self.text_standard_selector)]
        ages = correct_sample_ages(data_to_update,chosen_std,self.text_standard_selector)
        if self.output_data is None:
            self.output_data = ages
        else:
            self.output_data = self.output_data.append(ages,ignore_index=True)
        # self.input_data = self.input_data.update(ages) # will need to create blank columns on inputdata
        # self.input_data = ages # will need to create blank columns on inputdata
        
    @pn.depends('output_data')
    def export_data(self,event=None):
        self.output_data.to_excel('output_lasertramZ_ages.xlsx')

reduce_ages = finalize_ages(name='Reduce Ages')

In [30]:
pn.extension()

# fixed, stretch_width, stretch_height, stretch_both, scale_width, scale_height, scale_both, None]

# app = pn.Row(pn.Column(pn.Row(reduce_ages.call_TW
#                              ),
#                        pn.Row(reduce_ages._update_data_widget
#                              )
#                       ),
#              pn.Column(pn.Row(pn.WidgetBox(pn.Param(reduce_ages.param))
#                              )
#                       )
#             )
width_ratios=[10,5]
grid_layout = pn.GridSpec(sizing_mode='scale_both')
# grid_layout = pn.GridSpec()

grid_layout[0,0] = pn.Column(reduce_ages.call_TW)
grid_layout[1,0] = pn.Row(reduce_ages._update_data_widget)

grid_layout[0,1] = pn.Row(pn.WidgetBox(pn.Param(reduce_ages.param,
                                         # widgets={'regression_buttons': pn.widgets.CheckBoxGroup,
                                         #          'ratio_buttons': pn.widgets.CheckBoxGroup,
                                         #          'export_data_button': pn.widgets.Button(name='DDDT!',button_type='success')}
                                               )
                                      )
                         )

grid_layout.show()

Launching server at http://localhost:57607


/var/folders/h0/nxt863516yb3cyz22ybjvvm80000gn/T/ipykernel_40906/1180569260.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['207Pb/206Pbr'] = pts_pb_r
/var/folders/h0/nxt863516yb3cyz22ybjvvm80000gn/T/ipykernel_40906/1180569260.py:211: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['f'] = f
/var/folders/h0/nxt863516yb3cyz22ybjvvm80000gn/T/ipykernel_40906/1180569260.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind